# Chapter 2 — Algebra of Matrices (Colab Companion)

This notebook provides **Python helpers and demos** for core matrix-algebra topics:

- Matrix addition, scalar multiplication, and **matrix product**
- **Conformability** rules and common pitfalls
- **Transpose** rules; **symmetric** and **skew-symmetric** matrices
- **Inverse** matrices, uniqueness, and the identity (when it exists)
- **Orthogonal** matrices: `A.T @ A = I`
- **Powers** of a matrix and **polynomials in a matrix** \(Horner evaluation; `p(A)` commutes with `A`\)
- **Noncommutativity** (typically `AB != BA`) and **zero divisors** (nonzero `A,B` with `AB = 0`)

> Most checks are performed numerically with NumPy; for exact arithmetic and proofs we also include SymPy.

In [1]:
# Setup (NumPy/SymPy). In Colab these are typically available.
# If needed, you can uncomment the next line to install/upgrade:
# !pip -q install numpy sympy

import numpy as np
import sympy as sp

np.set_printoptions(suppress=True, linewidth=120)
sp.init_printing(use_unicode=True)

## 1) Shapes & conformability

Helpers to check whether a product `AB` is defined and to perform safe multiplication with clear errors.

In [2]:
def is_conformable(A, B):
    """Return True iff the product A @ B is defined."""
    return A.shape[1] == B.shape[0]

def mmul(A, B):
    """Matrix-multiply with a helpful error if shapes are incompatible."""
    if not is_conformable(A, B):
        raise ValueError(f"Not conformable: A is {A.shape}, B is {B.shape} (need A.shape[1] == B.shape[0])")
    return A @ B

# Demo
if __name__ == "__main__":
    A = np.ones((2,3))
    B = np.ones((3,4))
    C = np.ones((4,2))
    print("is_conformable(A,B)?", is_conformable(A,B))
    print("is_conformable(B,C)?", is_conformable(B,C))
    print("Shapes OK -> (A@B)@(C)?", is_conformable(A@B, C))

is_conformable(A,B)? True
is_conformable(B,C)? True
Shapes OK -> (A@B)@(C)? True


## 2) Algebraic properties: quick spot checks

We numerically verify associativity of multiplication and distributivity over addition on random integer matrices (where shapes allow).

In [5]:
rng = np.random.default_rng(0)

def rand_int_matrix(m, n, low=-3, high=4):
    return rng.integers(low, high, size=(m,n))

def check_associativity(m=3, k=4, n=2, p=5, trials=5):
    """Check (AB)C == A(BC) for random A∈R^{m×k}, B∈R^{k×n}, C∈R^{n×p}."""
    ok = True
    for _ in range(trials):
        A, B, C = rand_int_matrix(m,k), rand_int_matrix(k,n), rand_int_matrix(n,p)
        left  = (A @ B) @ C
        right = A @ (B @ C)
        if not np.allclose(left, right):
            ok = False; break
    return ok

def check_distributivity(m=3, k=4, n=2, trials=5):
    """Check A(B+C) == AB + AC and (A+B)C == AC + BC."""
    ok_left = ok_right = True
    for _ in range(trials):
        # Check A(B+C) = AB + AC; A: (m,k), B: (k,n), C: (k,n)
        A_left = rand_int_matrix(m,k)
        B_left = rand_int_matrix(k,n)
        C_left = rand_int_matrix(k,n)
        ok_left  &= np.allclose(A_left @ (B_left + C_left), A_left @ B_left + A_left @ C_left)

        # Check (A+B)C = AC + BC; A: (m,k), B: (m,k), C: (k,n)
        A_right = rand_int_matrix(m,k)
        B_right = rand_int_matrix(m,k)
        C_right = rand_int_matrix(k,n)
        ok_right &= np.allclose((A_right + B_right) @ C_right, A_right @ C_right + B_right @ C_right)

    return ok_left and ok_right

if __name__ == "__main__":
    print("Associativity holds? ", check_associativity())
    print("Distributivity holds? ", check_distributivity())

Associativity holds?  True
Distributivity holds?  True


## 3) Noncommutativity and zero divisors

`AB = BA` rarely holds unless under special structure. Also, nonzero matrices can multiply to the zero matrix.

In [8]:
def noncommuting_example():
    A = np.array([[0,1],[0,0]])
    B = np.array([[0,0],[1,0]])
    return A, B, A @ B, B @ A

def zero_divisors_example():
    A = np.array([[1,0],[0,0]])
    B = np.array([[0,0],[0,1]])
    return A, B, A @ B  # equals 0 though A,B are nonzero

if __name__ == "__main__":
    A, B, AB, BA = noncommuting_example()
    print(f"Noncommuting example:")
    print(f"A=\n{A}")
    print(f"B=\n{B}")
    print(f"A@B=\n{AB}")
    print(f"B@A=\n{BA}")
    A, B, AB = zero_divisors_example()
    print(f"\nZero divisors example (A,B ≠ 0 but A@B = 0):")
    print(f"A=\n{A}")
    print(f"B=\n{B}")
    print(f"A@B=\n{AB}")

Noncommuting example:
A=
[[0 1]
 [0 0]]
B=
[[0 0]
 [1 0]]
A@B=
[[1 0]
 [0 0]]
B@A=
[[0 0]
 [0 1]]

Zero divisors example (A,B ≠ 0 but A@B = 0):
A=
[[1 0]
 [0 0]]
B=
[[0 0]
 [0 1]]
A@B=
[[0 0]
 [0 0]]


## 4) Transpose rules; symmetric & skew-symmetric matrices

We verify `(AB)^T = B^T A^T`, `(kA)^T = k A^T`, and classify symmetric (`A.T == A`) vs skew-symmetric (`A.T == -A`).

In [9]:
def transpose_rules_demo():
    A = rand_int_matrix(3,4)
    B = rand_int_matrix(4,2)
    k = 5
    rule1 = np.allclose((A @ B).T, B.T @ A.T)
    rule2 = np.allclose((k*A).T, k*A.T)
    return rule1, rule2

def is_symmetric(A, tol=0.0):
    return np.allclose(A.T, A, atol=tol)

def is_skew_symmetric(A, tol=0.0):
    return np.allclose(A.T, -A, atol=tol)

if __name__ == "__main__":
    r1, r2 = transpose_rules_demo()
    print("(AB)^T = B^T A^T ? ", r1)
    print("(kA)^T = k A^T ?   ", r2)
    S = np.array([[2,1],[1,3]])
    K = np.array([[0,2],[-2,0]])
    print("S symmetric? ", is_symmetric(S))
    print("K skew-symmetric? ", is_skew_symmetric(K))

(AB)^T = B^T A^T ?  True
(kA)^T = k A^T ?    True
S symmetric?  True
K skew-symmetric?  True


## 5) Inverses (when they exist)

We test uniqueness, product rule `(AB)^{-1} = B^{-1} A^{-1}`, and basic identities. We use SymPy for exact checks when possible.

In [10]:
def invertible_sp(A):
    """Return SymPy inverse if invertible, else None."""
    M = sp.Matrix(A)
    if M.det() != 0:
        return np.array(M.inv(), dtype=float)
    return None

def inverse_product_rule_demo():
    A = rand_int_matrix(3,3, low=-3, high=3)
    B = rand_int_matrix(3,3, low=-3, high=3)
    # Make sure they are invertible by adding identity if necessary
    MA = sp.Matrix(A); MB = sp.Matrix(B)
    if MA.det() == 0: A = A + np.eye(3, dtype=int)
    if MB.det() == 0: B = B + np.eye(3, dtype=int)
    invA = np.array(sp.Matrix(A).inv(), dtype=float)
    invB = np.array(sp.Matrix(B).inv(), dtype=float)
    left  = np.array(sp.Matrix(A) * sp.Matrix(B), dtype=float)
    right = invB @ invA
    return np.allclose(np.linalg.inv(left), right)

if __name__ == "__main__":
    print("(AB)^{-1} = B^{-1}A^{-1} ? ", inverse_product_rule_demo())

(AB)^{-1} = B^{-1}A^{-1} ?  True


## 6) Orthogonal matrices

`A` is orthogonal iff `A.T @ A = I`. We generate examples using 2D rotations and via QR factorization.

In [11]:
def rotation2d(theta):
    c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s],[s, c]])

def is_orthogonal(A, tol=1e-10):
    I = np.eye(A.shape[0])
    return np.allclose(A.T @ A, I, atol=tol) and np.allclose(A @ A.T, I, atol=tol)

def random_orthogonal(n, rng=np.random.default_rng(1)):
    Q, _ = np.linalg.qr(rng.standard_normal((n,n)))
    # Ensure det(Q) = +1 (optional): flip one column if det negative
    if np.linalg.det(Q) < 0:
        Q[:,0] = -Q[:,0]
    return Q

if __name__ == "__main__":
    R = rotation2d(np.pi/7)
    print("Rotation orthogonal? ", is_orthogonal(R))
    Q = random_orthogonal(4)
    print("Random Q orthogonal? ", is_orthogonal(Q))

Rotation orthogonal?  True
Random Q orthogonal?  True


## 7) Matrix powers and polynomials p(A)

We implement Horner's method to evaluate `p(A) = c0 I + c1 A + ... + ck A^k` efficiently, and verify that `p(A)` **commutes** with `A`.

In [12]:
def mat_poly(A, coeffs):
    """Evaluate p(A) with Horner's rule.
    coeffs = [c0, c1, ..., ck] (ascending powers).
    """
    n = A.shape[0]
    P = np.zeros_like(A, dtype=float)
    I = np.eye(n)
    # Horner from highest to lowest: (((ck)A + ck-1)A + ... )A + c0
    # Implemented as: P = ck*I; for i in reversed(range(k)): P = A@P + ci*I
    ck = coeffs[-1]
    P = ck * I
    for c in reversed(coeffs[:-1]):
        P = A @ P + c * I
    return P

def commutes_with_A(A, P):
    return np.allclose(A @ P, P @ A)

if __name__ == "__main__":
    A = np.array([[0,1],[0,0]], dtype=float)  # nilpotent Jordan block
    coeffs = [1, 0, 2]  # p(A) = I + 2 A^2
    P = mat_poly(A, coeffs)
    print("p(A) commutes with A? ", commutes_with_A(A, P))

p(A) commutes with A?  True


## 8) (Bonus) Block matrices: product & transpose

A small helper to multiply conformable 2×2 block matrices and to verify `(AB)^T = B^T A^T` blockwise.

In [13]:
def blocks_2x2(A, B, C, D):
    """Pack 4 blocks into a 2x2 block matrix."""
    top = np.hstack((A, B))
    bot = np.hstack((C, D))
    return np.vstack((top, bot))

def block_multiply(X11, X12, X21, X22, Y11, Y12, Y21, Y22):
    """Multiply two 2x2 block matrices (assuming inner shapes match)."""
    Z11 = X11 @ Y11 + X12 @ Y21
    Z12 = X11 @ Y12 + X12 @ Y22
    Z21 = X21 @ Y11 + X22 @ Y21
    Z22 = X21 @ Y12 + X22 @ Y22
    return Z11, Z12, Z21, Z22

if __name__ == "__main__":
    A,B,C,D = [rand_int_matrix(2,2) for _ in range(4)]
    E,F,G,H = [rand_int_matrix(2,2) for _ in range(4)]
    Z = block_multiply(A,B,C,D, E,F,G,H)
    # Check shape of assembled product against dense multiplication
    X = blocks_2x2(A,B,C,D); Y = blocks_2x2(E,F,G,H)
    Z_full = blocks_2x2(*Z)
    print("Block product matches dense? ", np.allclose(Z_full, X @ Y))

Block product matches dense?  True


---

### How to use this notebook

1. Run the **Setup** cell.
2. Use **`is_conformable`** and **`mmul`** to validate products.
3. Try **`check_associativity`** and **`check_distributivity`** for quick property checks.
4. Explore **noncommutativity** and **zero divisors** via the provided examples.
5. Verify **transpose rules** and classify **symmetric/skew-symmetric** matrices.
6. Test **inverse** identities with SymPy exact arithmetic.
7. Build **orthogonal** matrices (`rotation2d`, `random_orthogonal`) and verify `A.T @ A = I`.
8. Evaluate **polynomials in a matrix** using **`mat_poly`** and confirm `p(A)` commutes with `A`.
9. (Optional) Use **block** helpers to experiment with 2×2 block algebra.